In [ ]:
import django, sys, os
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from scipy.sparse import find

import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from MulticoreTSNE import MulticoreTSNE as mTSNE
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import coverage_error, label_ranking_average_precision_score, label_ranking_loss
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2
import pickle
import scipy.sparse
import datetime
import scipy.stats as st
import scoping.utils.learning_utils as lu
import matplotlib

django.db.connection.close()


In [ ]:
pid = 178
qid = 8145
qid = 8609
q = Query.objects.get(pk=qid)
qs = Query.objects.filter(pk__in=[8145,7368,8272, 8696, 8609, 8696])

dids = set(Doc.objects.filter(query__in=qs).values_list('pk', flat=True))
p = Project.objects.get(pk=pid)

print(len(dids))

# Get documents from ar5 database, excluding those that look at alternative hypotheses than climate-impacts

In [ ]:
ar5_docs = Doc.objects.filter(query=6956, content__iregex="\w").exclude(
    pk__in=[
        1461495,1626875, 1622783,1627593,1627701,2093363,
        2090005,1627734,2089982, 2090013,2093360,2089960,
        1627733, 2090004, 2089967, 2089944, 2091676,2089959,  
        1514856, 2091680, 2089950
    ]
)

ar5_ids = set(ar5_docs.values_list('pk',flat=True))

ar5_docs.count()

In [ ]:
import statistics

def mode_na(x):
    try:
        return statistics.mode([y for y in x if y is not np.NaN])
    except:
        return np.NaN
    
columns = [
    'id',
    'content',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

# Majority ratings
dos = DocOwnership.objects.filter(
    query__in=qs, 
    relevant__gt=0,
    doc__content__iregex='\w'
)
dodf = pd.DataFrame.from_dict(dos.order_by('finish').values(
    'id','doc_id','user__username','relevant'
))
dodf.loc[dodf['relevant']==2,'relevant'] = 0
dodf.loc[dodf['relevant']==3,'relevant'] = np.NaN

gdf = dodf.groupby('doc_id').agg(lambda x: mode_na(x)).reset_index()[['doc_id','relevant']].rename(columns={"doc_id":"id"})
gdf['seen'] = 1
gdf['ar5'] = 0
gdf.loc[pd.isna(gdf['relevant']),'seen'] = np.NaN

# Ar5 docs
ar5_df = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=ar5_ids).values('id')
)
ar5_df['ar5'] = 1
ar5_df['seen'] = 1
ar5_df['relevant'] = 1
ar5_df.head()

seen_df = pd.concat([ar5_df,gdf])
ddf = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=seen_df['id']).values(*columns)
)
seen_df = pd.merge(ddf,seen_df)

r_tag_ids = Tag.objects.get(query__project=p,title="random sample 1").doc_set.values_list('pk',flat=True)
seen_df['random_sample'] = np.where(seen_df['id'].isin(r_tag_ids),1,0)

physical_tags = Tag.objects.filter(query__in=qs,id__gt=5177)
pdos = DocOwnership.objects.filter(tag__in=physical_tags,relevant__gt=0).values_list('doc__id',flat=True)
seen_df['physical_tags'] = np.where(seen_df['id'].isin(pdos),1,0)

seen_df = seen_df[pd.notna(seen_df['seen'])]

print(seen_df.shape)
seen_df.head()


## Get AR5 categories

In [ ]:
cats = Category.objects.filter(project=p,level=12)
early_ids = set(DocOwnership.objects.filter(
    query__in=qs,tag__lt=4765
).values_list('doc__pk',flat=True))
ldf = pd.DataFrame.from_dict(list(Doc.objects.filter(pk__in=set(ar5_ids)|early_ids).filter(
    category__in=cats
).values('id','category__level','category__name')))
ldf['cname'] =  ldf['category__level'].astype(str) + " - " + ldf['category__name']
ldf = ldf.drop(columns=['category__level','category__name'])
ldf['v'] = 1

cat_map = ldf.pivot(index='id',columns="cname", values="v")
cat_map['id'] = cat_map.index
cat_map = cat_map.reset_index(drop=True)

seen_df = pd.merge(seen_df, cat_map, how="left").fillna(0)

## Get majority category decision

In [ ]:
dudf = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        doc__pk__in=seen_df['id'],category__project=p
    ).values('user__username','doc__id','category__level','category__name')
)
dudf['v'] = 1
dudf['cname'] =  dudf['category__level'].astype(str) + " - " + dudf['category__name']#
dudf = dudf.drop(columns=['category__level','category__name'])
dudf_wide = dudf.pivot_table(index=['doc__id','user__username'],columns="cname", values="v").fillna(0)
print(dudf_wide.shape)
gudf = dudf_wide.groupby(['doc__id']).agg(lambda x: st.mode(x)[0][0]).reset_index()
dudf_wide_r = dudf_wide.reset_index()
dudf_wide_r.columns = ["doc__id",'user__username'] + list(dudf_wide.columns)
dudf_expanded = dudf_wide_r.melt(id_vars=["doc__id","user__username"]).head()
print(gudf.shape)
gudf.tail()

In [ ]:
from alterations import postfix_data

seen_df = pd.merge(seen_df, gudf.rename(columns={"doc__id":"id"}), how="left").fillna(0)
print(seen_df.shape)

seen_df = postfix_data(seen_df)
seen_df.to_csv('../data/0_labelled_documents.csv', index=False)
seen_df.head()

In [ ]:
unseen_docs = Doc.objects.filter(
    id__in=dids,
    content__iregex='\w',
).exclude(pk__in=seen_df['id']).distinct('id')

unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
unseen_df = unseen_df[columns]
unseen_df['relevant'] = 0
unseen_df['seen'] = 0
unseen_df['random_sample'] = 0

print(unseen_df.shape)
unseen_df.to_csv('../data/0_unlabelled_documents.csv', index=False)
unseen_df.head()

In [ ]:
django.db.connection.close()
df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)
y = df['relevant']

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
new_index = df[(df['seen']==1) & (df['ar5']==0)].index
rel_index = df[df['relevant']==1].index

r_index = df[df["random_sample"]==1].index
print(df.shape)
df.head()